<a href="https://colab.research.google.com/github/IamLIAcastaP/AI-Data-Science-/blob/main/Pearson_Coseno_OK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import string
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from decimal import Decimal
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

#Cargando el dataframe de los videos
video_df = pd.read_csv('/content/drive/MyDrive/Dataframe/GBvideos.csv')
#Eliminando las columnas que no voy a utiliza
video_df = video_df.drop(['trending_date','channel_title', 'category_id', 'publish_time','views','comment_count', 'thumbnail_link', 'comments_disabled','ratings_disabled','video_error_or_removed', 'description'], axis=1)

#limpiando la columna del titulo del videos creando una columna nueva con los titulos limpios
video_df['Ctitle']= video_df['title'].str.replace('[{}]'.format(string.punctuation),'')

#Cargando el dataframe de los usuarios y seleccionando solo el 0.0001%
user_df = pd.read_csv('/content/drive/MyDrive/Dataframe/RatUser.csv')
user_dfSample = user_df.sample(frac=0.001)

#Creando un susario random a quien recomendar las peliculas
#Tomando las primeras 10 filas del dataframe original de videos
video_dfSample = video_df.iloc[:10,:]
#tomando de la muestra la columna de los titulos limpios y agregando al usuario random
userLikes= pd.DataFrame()
userLikes['Ctitle']= video_dfSample['Ctitle']
userLikes['video_id']= video_dfSample['video_id']
userLikes['Rating']= user_df['Rating']

#Filtrando los usuarios que han visto los videos y guardandolos
userSubset = user_df[user_df['video_id'].isin(userLikes['video_id'].tolist())]
#Agrupandolos por usuario
userSubsetGroup=userSubset.groupby(['User_ID'])


#Organizando los usuarios de forma que los usuarios con mas peliculas en comun tengan prioridad
userSubsetGroup = sorted( userSubsetGroup, key=lambda x: len(x[1]), reverse=True)

from os import name
#Calculando la correlacion de pearson en un diccionario entre la entrada del susario y el grupo para almacenarlo
#diccionario
pearsonCorrelationDic = {}

#Para cada grupo de usuarios en nuestro subconjunto
for name, group in userSubsetGroup:
     #Ingresando el usuario actual y el ingresado sin que los valores se mezclen
     group = group.sort_values(by='video_id')
     userLikes = userLikes.sort_values(by='video_id')
     nRatings = len(group)
     temp_df = userLikes[userLikes['video_id'].isin(group['video_id'].tolist())]
     tempRatingList = temp_df['Rating'].tolist()
     tempGroupList = group['Rating'].tolist()
     #Calculando la correlacion de Pearson entre dos usuarios
     Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList), 2)/float(nRatings)
     Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList), 2)/ float(nRatings)
     Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

     if Sxx != 0 and Syy != 0:
       pearsonCorrelationDic[name] = Sxy/np.sqrt(Sxx*Syy)
     else:
       pearsonCorrelationDic[name] = 0

pearsonCorrelationDic.items()
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDic, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['User_ID'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))

#Obteniendo los primeron 50 usuarios mas parecidos a los que se ingresaron
topUsers = pearsonDF.sort_values(by = 'similarityIndex', ascending = False)[0:50]

#Recomendamos las peliculas al usuario de entrada puntuando a los usuarios elegidos para todas las peliculas
topUsersRating = topUsers.merge(user_df, left_on = 'User_ID', right_on= 'User_ID', how='inner')

#Multiplicando la similitud de los puntajes de los usuarios
topUsersRating['weightedRating'] = topUsersRating['similarityIndex'] * topUsersRating['Rating']

#Aplicando una suma a los topUsers luego de agruparlos por el id
tempTopUsersRating = topUsersRating.groupby(['video_id'])
tempTopUsersRating = tempTopUsersRating.sum()[['similarityIndex', 'weightedRating']]
tempTopUsersRating.columns =  ['sum_similarityIndex' , 'sum_weightedRating']

#Creando un dataframe vacio y agregandole el promedio ponderado
recommendation_df = pd.DataFrame()
recommendation_df['weighted average recommendation score'] = tempTopUsersRating ['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['video_id'] = tempTopUsersRating.index

#Ordenando y ver las primeras 20 peliculas que el algoritmo recomendo
recommendation_df = recommendation_df.sort_values(by = 'weighted average recommendation score' , ascending=False)

DF = pd.DataFrame()
DF= video_df.loc[video_df['video_id'].isin(recommendation_df.head(10)['video_id'].tolist())]
DF = DF.drop_duplicates (['Ctitle'])
DF


<ipython-input-1-1982f430ffe5>:18: FutureWarning: The default value of regex will change from True to False in a future version.
  video_df['Ctitle']= video_df['title'].str.replace('[{}]'.format(string.punctuation),'')
<ipython-input-1-1982f430ffe5>:40: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup = sorted( userSubsetGroup, key=lambda x: len(x[1]), reverse=True)


,video_id,title,tags,likes,dislikes,Ctitle
27,F62QpaPWXDI,THE CHEAPEST & BEST EYESHADOWS?,"jamie genevieve|""bblogger""|""youtuber""|""makeup""...",5029,81,THE CHEAPEST BEST EYESHADOWS
56,P4ohTV_28Oc,Taylor Swift Saturday Night Live Behind The Sc...,"Taylor swift|""snl""|""funny Taylor""|""saturday ni...",915,12,Taylor Swift Saturday Night Live Behind The Sc...
83,KdFRkillEfM,*INSANE* I SPENT THE NIGHT IN A SKI CENTRE!,"illegal|""police""|""insane""|""snow""|""snozone""|""sk...",4517,65,INSANE I SPENT THE NIGHT IN A SKI CENTRE
106,3iXwXYflCSI,If only every weekend could be like this | Emi...,"emilyhart|""emily hartridge""|""vlog""|""family""|""s...",812,15,If only every weekend could be like this Emil...
184,tpyQLIpv4Yk,POP SONGS TURNED OPERA,"colleen ballinger|""colleen""|""ballinger""|""psych...",48822,408,POP SONGS TURNED OPERA
201,VsYmwBOYfW8,Mean Tweets – Jimmy Kimmel Edition,"jimmy|""jimmy kimmel""|""jimmy kimmel live""|""late...",35529,1705,Mean Tweets – Jimmy Kimmel Edition
202,A5TPOKoE1_M,CHRISTMAS ADVERTS & HOW WE MET,"zoe sugg|""zoella""|""zoe""|""vlog""|""vlogging""|""vlo...",33256,994,CHRISTMAS ADVERTS HOW WE MET
223,gKo7V7nxvqE,Harry Styles returns to The X Factor stage! | ...,"the x factor|""x factor""|""X factor UK""|""x facto...",51873,586,Harry Styles returns to The X Factor stage Li...
402,jG1JY0rt2Os,Fall Out Boy - HOLD ME TIGHT OR DON’T,"Fall Out Boy|""Hold Me Tight Or Don’t""|""Hold Me...",114661,2509,Fall Out Boy HOLD ME TIGHT OR DON’T
409,vIVaLtJmyPs,UNHhhh Ep 68: Goodbye Barbara w/ Trixie Mattel...,"68|""barbara""|""ep""|""goodbye""|""katya""|""mattel""|""...",25407,228,UNHhhh Ep 68 Goodbye Barbara w Trixie Mattel ...


In [ ]:
#Singularidad Coseno
DataF= pd.read_csv('/content/drive/MyDrive/Dataframe/GBvideos.csv')

#base de dato pequena
VideosDF = DataF.sample(frac=0.1)
VideosDF = VideosDF.reset_index(drop=True)

#para quitar los signos que tenga la base de dato
VideosDF['ntags'] = VideosDF['tags'].str.replace('[{}]'.format(string.punctuation),' ')

# palabras por numeros
tfidf= TfidfVectorizer(stop_words='english')
VideosDF['ntags']=VideosDF['ntags'].fillna('')

#la matriz de transformacion
tfidf_matrix = tfidf.fit_transform(VideosDF['ntags'])

#simulitud de coseno
cosine_sim=linear_kernel(tfidf_matrix,tfidf_matrix)
#cosine_similarity(tfidf_matrix,tfidf_matrix)

#elimar los duplicados
indice= pd.Series(VideosDF.index, index=VideosDF['title']).drop_duplicates()
def recomendacion(title,cosine_sim=cosine_sim):
  idx=indice[title]
  sim_scores=list(enumerate(cosine_sim[idx]))
  sim_scores=sorted(sim_scores,key=lambda x:x[1], reverse=True)
  sim_scores= sim_scores[1:5]
  movie_indices=[i[0] for i in sim_scores]
  return VideosDF['title'].iloc[movie_indices]
#para seleccionar a partir de quien vamos a recomendar
titulo=str(VideosDF.iloc[1]['title'])

print("Tu seleccionaste: "+ titulo+" y las recomendaciones son:  " )
print (recomendacion(titulo))


Tu seleccionaste: WHAT I BOUGHT LAST MONTH: AUTUMN HAUL | Lily Pebbles y las recomendaciones son:  
24                        THE SWEATIEST I'VE EVER BEEN!
23    GET READY WITH ME + LILY! (we're so immature bye)
32                      Sexy Darth Vader | Lex Croucher
40        DRESSED UP IN ALL DOLCE AND GABBANA | VLOG 51
Name: title, dtype: object


<ipython-input-2-9216f5e236d9>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  VideosDF['ntags'] = VideosDF['tags'].str.replace('[{}]'.format(string.punctuation),' ')
